## Automatização de Relatórios Utilizando Dados GIS Vetoriais

### AUTOR: ADENILSON SILVA

# EXTRAÇÃO, TRANSFORMAÇÃO E CARRAGAMENTO DE DADOS

####  1 - Importando bibliotecas e criando funções

In [1]:
import os  # Importa o módulo 'os' para lidar com operações do sistema de arquivos
import geopandas as gpd  # Biblioteca para trabalhar com dados geoespaciais, como shapefiles e GeoJSON
import pandas as pd  # Usada para trabalhar com dados em formato de tabela (DataFrame)
import fiona  # Utilizada para ler e escrever arquivos geoespaciais (formato shapefile, por exemplo)
import chardet  # Usado para detectar a codificação de arquivos de texto
import json  #  Biblioteca para trabalhar com dados no formato JSON
from shapely.wkt import loads  # Importa a função 'loads' para converter strings WKT em objetos geométricos
from google.cloud import bigquery  # Cliente para interagir com o Google BigQuery, serviço de data warehouse na nuvem
from google.oauth2 import service_account  # Para autenticação com credenciais (chave JSON) em serviços do Google Cloud

In [2]:
def listar_shapefiles(diretorio_base):
    """
    Lista todos os arquivos shapefile (.shp) encontrados em um diretório base e suas subpastas,
    estruturando a hierarquia dos diretórios em colunas de um DataFrame pandas.

    Parâmetros:
    - diretorio_base (str): O caminho do diretório onde a busca por shapefiles deve começar.

    Retorna:
    - pandas.DataFrame: Um DataFrame contendo informações sobre cada shapefile encontrado. As colunas incluem:
        - 'id': Um identificador único para cada shapefile (baseado no índice do DataFrame).
        - 'caminho': O caminho completo do arquivo shapefile.
        - 'nivel_1', 'nivel_2', ..., 'nivel_8': As partes da hierarquia do caminho do arquivo, divididas em até 8 níveis.
                                               Se o caminho tiver menos de 8 níveis, as colunas restantes são preenchidas com None.
        - 'shapefile_nome': O nome do arquivo shapefile.

    Exemplo de uso:
    Suponha que você tenha a seguinte estrutura de diretórios:
    diretorio_base/
        nivel_1/
            nivel_2/
                arquivo1.shp
                arquivo2.shp
        nivel_3/
            arquivo3.shp

    Ao chamar listar_shapefiles("diretorio_base"), a função retornará um DataFrame com as seguintes informações:

    | id | caminho                                       | nivel_1 | nivel_2 | nivel_3 | nivel_4 | nivel_5 | nivel_6 | nivel_7 | nivel_8 | shapefile_nome |
    |----|-----------------------------------------------|---------|---------|---------|---------|---------|---------|---------|----------------|
    | 0  | diretorio_base/nivel_1/nivel_2/arquivo1.shp   | nivel_1 | nivel_2 | None    | None    | None    | None    | None    | None    | arquivo1.shp   |
    | 1  | diretorio_base/nivel_1/nivel_2/arquivo2.shp   | nivel_1 | nivel_2 | None    | None    | None    | None    | None    | None    | arquivo2.shp   |
    | 2  | diretorio_base/nivel_3/arquivo3.shp           | nivel_3 | None    | None    | None    | None    | None    | None    | None    | arquivo3.shp   |

    Observações:
    - A função utiliza os.walk do módulo os para percorrer recursivamente o diretório base e suas subpastas.
    - A hierarquia dos diretórios é limitada a 8 níveis. Se houver mais níveis, eles não serão incluídos nas colunas do DataFrame.
    - A função depende das bibliotecas os e pandas estarem instaladas e importadas corretamente.
    """
    registros = []
    for root, _, arquivos in os.walk(diretorio_base):
        for arquivo in arquivos:
            if arquivo.endswith(".shp"):
                caminho = os.path.join(root, arquivo)

                # Criando a hierarquia dividindo o caminho
                partes = caminho.replace(diretorio_base, "").strip(os.sep).split(os.sep)

                # Garantindo um número fixo de colunas para hierarquia (ajustável)
                niveis = partes[:-1]  # Tudo menos o arquivo shapefile
                while len(niveis) < 8:
                    niveis.append(None)

                registros.append([caminho] + niveis + [arquivo])

    colunas = ["caminho", "nivel_1", "nivel_2", "nivel_3", "nivel_4", "nivel_5", "nivel_6", "nivel_7", "nivel_8", "shapefile_nome"]
    df_hierarquia = pd.DataFrame(registros, columns=colunas)

    # Criando a coluna id baseada no índice
    df_hierarquia.insert(0, "id", df_hierarquia.index)

    return df_hierarquia

In [3]:
def carregar_geometrias(df_hierarquia):
    """
    Carrega geometrias de arquivos shapefile (.shp) e seus atributos, convertendo-os para formato GeoJSON.

    Parâmetros:
    - df_hierarquia (pandas.DataFrame): Um DataFrame contendo informações sobre arquivos shapefile,
      incluindo o caminho completo para cada arquivo. Este DataFrame deve ter pelo menos uma coluna
      chamada 'caminho'.

    Retorna:
    - pandas.DataFrame: Um DataFrame com duas colunas:
        - 'geometria_e_atributos': Dados GeoJSON representando as geometrias e atributos de cada shapefile.
        - 'id_diretorio': O índice do DataFrame de entrada, usado para identificar a origem dos dados.

    Descrição:
    Esta função itera sobre as linhas de um DataFrame fornecido, onde cada linha contém o caminho
    para um arquivo shapefile. A função tenta carregar cada shapefile, lida com possíveis problemas
    de codificação, converte as geometrias e atributos para GeoJSON e armazena os resultados em um
    novo DataFrame.

    Tratamento de Erros:
    - A função lida com erros de leitura de arquivos shapefile, tentando detectar a codificação correta
      caso a leitura direta falhe.
    - Se a conversão para GeoJSON falhar, a função imprime uma mensagem de erro e continua com o próximo arquivo.

    Conversão para WGS 84:
    - Garante que o sistema de referência de coordenadas (CRS) de cada GeoDataFrame seja WGS 84 (EPSG:4326).

    Exemplo de uso:
    Suponha que você tenha um DataFrame 'df_shapefiles' com uma coluna 'caminho' contendo os caminhos
    para seus arquivos shapefile.

    resultado = carregar_geometrias(df_shapefiles)
    print(resultado.head())

    Observações:
    - A função depende das bibliotecas pandas, geopandas, json, chardet e fiona estarem instaladas.
    - O GeoJSON resultante inclui tanto as geometrias quanto os atributos dos dados do shapefile.
    - A coluna 'id_diretorio' no DataFrame de saída corresponde ao índice do DataFrame de entrada,
      permitindo rastrear a origem dos dados.
    """
    geometrias_atributos = pd.DataFrame(columns=['geometria_e_atributos', 'id_diretorio'])
    for index, row in df_hierarquia.iterrows():
        caminho = row["caminho"]
        try:
            # Tentar carregar diretamente
            gdf = gpd.read_file(caminho)
        except Exception:
            try:
                # Detectar encoding e tentar novamente
                with open(caminho, "rb") as f:
                    resultado = chardet.detect(f.read(100))
                encoding_detectado = resultado["encoding"]
                with fiona.open(caminho, encoding=encoding_detectado) as src:
                    gdf = gpd.GeoDataFrame.from_features(src, crs=src.crs)
            except Exception as e:
                print(f"Erro ao processar {caminho}: {e}")
                continue
        # Garantir que CRS seja WGS 84
        if gdf.crs and gdf.crs.to_epsg() != 4326:
            gdf = gdf.to_crs(epsg=4326)
        try:
            gdf = gdf.apply(lambda col: col.astype(str) if col.dtype == 'datetime64[ms]' else col)
            geojson_result = json.loads(gdf.to_json(indent=4, ensure_ascii=False))
            geojson_result_str = json.dumps(geojson_result, ensure_ascii=False)
            linha = pd.DataFrame({'geometria_e_atributos': [geojson_result_str], 'id_diretorio': [index]})
            geometrias_atributos = pd.concat([geometrias_atributos, linha], ignore_index=True)
            df_hierarquia_geometrias_e_atributos = df_hierarquia.merge(geometrias_atributos, left_on='id', right_on='id_diretorio').drop(columns=['id_diretorio'])
            df_hierarquia_geometrias_e_atributos['id'] = df_hierarquia_geometrias_e_atributos['id'].astype('int64')
        except Exception as e:
            print(f"Erro ao converter geometrias para JSON em {caminho}: {e}")
    return df_hierarquia_geometrias_e_atributos

In [4]:
def criar_dataset_bigquery(caminho_chave, project_id, nome_dataset, localizacao="US"):
    """
    Cria um dataset no Google BigQuery com base em um arquivo de credenciais
    e retorna o cliente autenticado.

    Parâmetros:
    - caminho_chave: caminho para o arquivo JSON da conta de serviço.
    - project_id: ID do projeto no GCP.
    - nome_dataset: nome do dataset a ser criado.
    - localizacao: localização do dataset (ex: 'US', 'southamerica-east1', etc).

    Retorno:
    - client: instância autenticada de bigquery.Client
    """
    try:
        # Cria as credenciais e o cliente
        credentials = service_account.Credentials.from_service_account_file(caminho_chave)
        client = bigquery.Client(credentials=credentials, project=project_id)
        # Define e cria o dataset
        dataset_id = f"{client.project}.{nome_dataset}"
        dataset = bigquery.Dataset(dataset_id)
        dataset.location = localizacao
        dataset = client.create_dataset(dataset, exists_ok=True)
        print(f"Dataset '{dataset.dataset_id}' criado com sucesso na região '{localizacao}'.")
        return client
    except Exception as e:
        print(f"Erro ao criar o dataset: {e}")
        return None

In [5]:
def enviar_para_bigquery(dados, caminho_chave, projeto_id, dataset_id, tabela_id):
    '''
    Envia os dados de um DataFrame para uma tabela no Google BigQuery, substituindo os dados existentes.

    Parâmetros:
    - dados (DataFrame): DataFrame do pandas contendo os dados a serem carregados.
    - caminho_chave (str): Caminho para o arquivo de chave da conta de serviço do Google Cloud.
    - projeto_id (str): ID do projeto no Google Cloud.
    - dataset_id (str): Nome do dataset no BigQuery onde os dados serão armazenados.
    - tabela_id (str): Nome da tabela dentro do dataset onde os dados serão carregados.

    Observações:
    - A tabela será sobrescrita a cada execução (WRITE_TRUNCATE).
    - Apenas as colunas especificadas no schema serão carregadas.
    '''
    tabela_referencia = f"{dataset_id}.{tabela_id}"
    cliente = criar_dataset_bigquery(caminho_chave, projeto_id, dataset)    
    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("id", "int64"),
            bigquery.SchemaField("nivel_1", "STRING"),
            bigquery.SchemaField("nivel_2", "STRING"),
            bigquery.SchemaField("nivel_3", "STRING"),
            bigquery.SchemaField("nivel_4", "STRING"),
            bigquery.SchemaField("nivel_5", "STRING"),
            bigquery.SchemaField("nivel_6", "STRING"),
            bigquery.SchemaField("nivel_7", "STRING"),
            bigquery.SchemaField("nivel_8", "STRING"),
            bigquery.SchemaField("shapefile_nome", "STRING"),
            bigquery.SchemaField("geometria_e_atributos", "STRING"),
        ],
        write_disposition="WRITE_TRUNCATE", # sobrescreve a tabela (apaga os dados existentes antes de carregar novos)
    ) 
    job = cliente.load_table_from_dataframe(dados, tabela_referencia, job_config=job_config)
    job.result()
    print(f"Dados carregados na tabela {tabela_referencia}")

#### 2 - Lendo os arquivos e criando hierarquia

In [6]:
diretorio = "shapefiles"
df_hierarquia = listar_shapefiles(diretorio)
df_hierarquia.head(10)

,id,caminho,nivel_1,nivel_2,nivel_3,nivel_4,nivel_5,nivel_6,nivel_7,nivel_8,shapefile_nome
0,0,shapefiles\APAs\APA do Cafuringa\APA do Cafuri...,APAs,APA do Cafuringa,None,None,None,None,None,None,APA do Cafuringa.shp
1,1,shapefiles\APAs\APA do Descoberto\APA do Desco...,APAs,APA do Descoberto,None,None,None,None,None,None,APA do Descoberto.shp
2,2,shapefiles\APAs\APA do Gama Cabeça de Veado\AP...,APAs,APA do Gama Cabeça de Veado,None,None,None,None,None,None,APA do Gama Cabeça de Veado.shp
3,3,shapefiles\APAs\APA do Planalto Central\APA do...,APAs,APA do Planalto Central,None,None,None,None,None,None,APA do Planalto Central.shp
4,4,shapefiles\APAs\APA do São Bartolomeu\APA do S...,APAs,APA do São Bartolomeu,None,None,None,None,None,None,APA do São Bartolomeu.shp
5,5,shapefiles\APAs\APA IBRAM 2020\APA IBRAM 2020.shp,APAs,APA IBRAM 2020,None,None,None,None,None,None,APA IBRAM 2020.shp
6,6,shapefiles\Limites\Regiões Administrativas\Reg...,Limites,Regiões Administrativas,None,None,None,None,None,None,Regiões Administrativas.shp
7,7,shapefiles\Limites\Área Tombada\Área Tombada.shp,Limites,Área Tombada,None,None,None,None,None,None,Área Tombada.shp
8,8,shapefiles\PDOT\Conector Ambiental\Conector Am...,PDOT,Conector Ambiental,None,None,None,None,None,None,Conector Ambiental.shp
9,9,shapefiles\PDOT\Zoneamento\Zoneamento.shp,PDOT,Zoneamento,None,None,None,None,None,None,Zoneamento.shp


In [7]:
df_hierarquia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              33 non-null     int64 
 1   caminho         33 non-null     object
 2   nivel_1         33 non-null     object
 3   nivel_2         33 non-null     object
 4   nivel_3         14 non-null     object
 5   nivel_4         2 non-null      object
 6   nivel_5         0 non-null      object
 7   nivel_6         0 non-null      object
 8   nivel_7         0 non-null      object
 9   nivel_8         0 non-null      object
 10  shapefile_nome  33 non-null     object
dtypes: int64(1), object(10)
memory usage: 3.0+ KB


#### 3 - Lendos os arquivos _shapefiles_ e criando geodataframe com os dados das geometrias e atributos

**Observação:** 

Considerando que cada arquivo shapefile contém geometrias e atributos distintos, os dados foram consolidados em uma única coluna denominada "geometria_e_atributos". Esses dados foram convertidas para o formato JSON em forma de _string_, resultando em dados semiestruturados.

In [8]:
df_geoespacial = carregar_geometrias(df_hierarquia)
df_geoespacial.head()

,id,caminho,nivel_1,nivel_2,nivel_3,nivel_4,nivel_5,nivel_6,nivel_7,nivel_8,shapefile_nome,geometria_e_atributos
0,0,shapefiles\APAs\APA do Cafuringa\APA do Cafuri...,APAs,APA do Cafuringa,None,None,None,None,None,None,APA do Cafuringa.shp,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
1,1,shapefiles\APAs\APA do Descoberto\APA do Desco...,APAs,APA do Descoberto,None,None,None,None,None,None,APA do Descoberto.shp,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
2,2,shapefiles\APAs\APA do Gama Cabeça de Veado\AP...,APAs,APA do Gama Cabeça de Veado,None,None,None,None,None,None,APA do Gama Cabeça de Veado.shp,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3,3,shapefiles\APAs\APA do Planalto Central\APA do...,APAs,APA do Planalto Central,None,None,None,None,None,None,APA do Planalto Central.shp,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
4,4,shapefiles\APAs\APA do São Bartolomeu\APA do S...,APAs,APA do São Bartolomeu,None,None,None,None,None,None,APA do São Bartolomeu.shp,"{""type"": ""FeatureCollection"", ""features"": [{""i..."


In [9]:
df_geoespacial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     33 non-null     int64 
 1   caminho                33 non-null     object
 2   nivel_1                33 non-null     object
 3   nivel_2                33 non-null     object
 4   nivel_3                14 non-null     object
 5   nivel_4                2 non-null      object
 6   nivel_5                0 non-null      object
 7   nivel_6                0 non-null      object
 8   nivel_7                0 non-null      object
 9   nivel_8                0 non-null      object
 10  shapefile_nome         33 non-null     object
 11  geometria_e_atributos  33 non-null     object
dtypes: int64(1), object(11)
memory usage: 3.2+ KB


In [10]:
caminho_chave = "C:\Users\stefanini\projeto III\credenciais.json"
projeto_id = "uso-de-dados-gis-vetoriais"
dataset = "projeto_3"
tabela = "shapefile"
enviar_para_bigquery(df_geoespacial.drop(columns=['caminho']), caminho_chave, projeto_id, dataset, tabela)

Erro ao criar o dataset: [Errno 2] No such file or directory: 'C:/Users/stefanini/Desktop/Demandas/Automacao/projeto III/uso-de-dados-gis-vetoriais-0b65caeff0b0.json'


AttributeError: 'NoneType' object has no attribute 'load_table_from_dataframe'

### Dados sobre versão

- Python: 3.9.7
- pandas==2.2.3
- geopandas==1.0.1
- fiona==1.10.1
- chardet==4.0.0
- shapely==2.0.7
- google.cloud.bigquery = 3.31.0
- google-auth==2.38.0